In [29]:
import json
import torch
import transformers

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, TrainingArguments, Trainer


In [16]:
data = json.load(open('./TutorialVQAData/videos.json','r'))
train = json.load(open('./TutorialVQAData/train.json','r'))
test = json.load(open('./TutorialVQAData/test.json','r'))

In [17]:
df = {}
for i in range(len(data)):
    df[data[i]['video_id']] = data[i]

In [21]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [22]:
def get_answer(question, text):

    inputs = tokenizer(question, text, return_tensors='pt')
    #start_positions = torch.tensor([3])
    #end_positions = torch.tensor([7])
    #outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
    outputs = model(**inputs)
    loss = outputs.loss
    print(loss)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    tokens = inputs['input_ids'][0][torch.argmax(start_scores): torch.argmax(end_scores) + 1]
    answerTokens = tokenizer.convert_ids_to_tokens(tokens, skip_special_tokens=True)
    
    return torch.argmax(start_scores)-len(tokenizer(question)['input_ids']), torch.argmax(end_scores)+1 -len(tokenizer(question)['input_ids']), tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])), tokenizer.convert_tokens_to_string(answerTokens)

In [23]:
question, text = "what is jim henson?", "Jim Henson was a nice puppet. John Smith was a banker."
start, end, answerTokens, answer = get_answer(question, text)
print(start, end, answerTokens, answer)

None
tensor(3) tensor(6) [CLS] what is jim henson ? [SEP] jim henson was a nice puppet . john smith was a banker . [SEP] a nice puppet


In [26]:
def test_single(question_df, context):
    context = ' '.join(context['transcript'])
    question = question_df['question']
    start_true = question_df['answer_start']
    end_true = question_df['answer_end']
    
    start, end, answerTokens, answer = get_answer(question, context[:512])
    
    print('question: ', question)
    print('answer: ', answer)
    
    return start_true-start, end_true-end

In [27]:
for i in range(10):
    print(test_single(test[i], df[test[i]['video_id']]))

None
question:  how do i create a new action?
answer:  
(tensor(-111), tensor(-103))
None
question:  how do i change the text color of an image?
answer:  i want to create beautiful type
(tensor(5), tensor(3))
None
question:  what do you have to do to get a new layer?
answer:  take a look at this layers panel
(tensor(-70), tensor(-72))
None
question:  how to align objects to layers.
answer:  
(tensor(46), tensor(55))
None
question:  3
answer:  tutorial page or photos of your own
(tensor(-44), tensor(-51))
None
question:  why isn't quick select actually selecting the selected bit?
answer:  it actually selects too much , and it ends up selecting the entire document because there aren ' t very many tonal or color values between the green and the blue
(tensor(-76), tensor(-106))
None
question:  how can i keep cropped pixels?
answer:  by selecting the crop tool here in the tools panel
(tensor(-61), tensor(-64))
None
question:  how do i save my project?
answer:  you can use the sample images 

In [30]:
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

NameError: name 'small_train_dataset' is not defined